## **Chat Models**

In [4]:
from dotenv import load_dotenv
import os
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [7]:
from langchain.chat_models import init_chat_model
llm = init_chat_model("groq:qwen/qwen3-32b", temperature=0)

response = llm.invoke("test line")

In [8]:
response.content

'<think>\nOkay, the user sent "test line". That\'s pretty short. I need to figure out what they want. Maybe they\'re just checking if the system is working. Let me see. They might be testing the response time or if the AI is active. I should respond in a friendly way to acknowledge their test. Let me make sure to keep it open-ended in case they have more to say. Maybe something like, "Hello! It looks like you\'re testing the waters. How can I assist you today?" That should cover it. Let me check for any typos and ensure the tone is approachable.\n</think>\n\nHello! It looks like you\'re testing the waters. How can I assist you today? 😊'

In [9]:
type(response)

langchain_core.messages.ai.AIMessage

# **Tools**

In [10]:
from langchain.tools import tool

@tool
def write_email(to: str, subject: str, content: str) -> str:
    """Write and send email."""

    return f"Email sent to {to} with subject {subject} and content: {content}"

In [11]:
type(write_email)

langchain_core.tools.structured.StructuredTool

In [12]:
write_email.args

{'to': {'title': 'To', 'type': 'string'},
 'subject': {'title': 'Subject', 'type': 'string'},
 'content': {'title': 'Content', 'type': 'string'}}

## **Tools Binding**
- tool choice: `any`
    - Always have to tool calls from the available tools.

In [13]:
model_with_tools = llm.bind_tools(tools=[write_email], tool_choice="any", parallel_tool_calls=False)

output = model_with_tools.invoke("Draft a response to my MD sir about tomorrow's meeting")

In [14]:
output

AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user wants me to draft a response to their MD sir about tomorrow\'s meeting. Let me start by understanding what they need. They probably need a professional email that confirms their attendance or discusses the meeting agenda.\n\nFirst, I should check if there are any specific details they want included. The user hasn\'t provided specifics, so I\'ll have to make some assumptions. The email should have a clear subject line, like "Confirmation of Attendance for Tomorrow\'s Meeting" or something similar.\n\nThe content should be polite, mention the meeting date, maybe the time and location if known. Since it\'s to the MD, it\'s important to maintain a respectful tone. I should also include a placeholder for any additional details the user might want to add, like agenda points or action items.\n\nWait, the user might need to add personal notes or specific requests. I\'ll structure the email with a friendly opening, a 

In [16]:
output.tool_calls[0]

{'name': 'write_email',
 'args': {'content': 'Dear MD Sir,\n\nI hope this email finds you well. I would like to confirm my attendance for the meeting scheduled tomorrow. Please let me know if there are any specific agenda items or preparations required from my side.\n\nLooking forward to your guidance.\n\nBest regards,\n[Your Name]',
  'subject': "Confirmation of Tomorrow's Meeting",
  'to': 'md@example.com'},
 'id': 'ybwkxd3k6',
 'type': 'tool_call'}

In [17]:
args = output.tool_calls[0]["args"]
args

{'content': 'Dear MD Sir,\n\nI hope this email finds you well. I would like to confirm my attendance for the meeting scheduled tomorrow. Please let me know if there are any specific agenda items or preparations required from my side.\n\nLooking forward to your guidance.\n\nBest regards,\n[Your Name]',
 'subject': "Confirmation of Tomorrow's Meeting",
 'to': 'md@example.com'}

In [18]:
result = write_email.invoke(args)

In [20]:
print(result)

Email sent to md@example.com with subject Confirmation of Tomorrow's Meeting and content: Dear MD Sir,

I hope this email finds you well. I would like to confirm my attendance for the meeting scheduled tomorrow. Please let me know if there are any specific agenda items or preparations required from my side.

Looking forward to your guidance.

Best regards,
[Your Name]


# ***LangGraph***

In [21]:
from typing import TypedDict
from langgraph.graph import StateGraph, START, END

class StateSchema(TypedDict):
    request: str
    email: str

workflow = StateGraph(StateSchema)